In [1]:
!pip install keras-tuner
import sys
import pickle
import argparse
import pandas as pd
import numpy as np
from tensorflow import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras.layers import Dropout
from keras.utils.np_utils import to_categorical
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
import kerastuner
from kerastuner.tuners import RandomSearch, Hyperband
from kerastuner import HyperModel
import keras.metrics

     |████████████████████████████████| 61kB 2.0MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.1-cp36-none-any.whl size=73200 sha256=50bd362132f4276d671c19d00be947643c09b75fb6a7d1abf183e63ae837a7a8
  Stored in directory: /root/.cache/pip/wheels/b9/cc/62/52716b70dd90f3db12519233c3a93a5360bc672da1a10ded43
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp36-none-any.whl size=15356 sha256=4e5a83b42b9c9f84e83f0f3d6c61af2ced5fdeb819930d9c7b1f273149420406
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


Using TensorFlow backend.


In [0]:
df = pickle.load(open('reviews_20000_processed.p', 'rb'))
vocab = pickle.load(open('reviews_20000_vocab.p', 'rb'))

In [3]:
model_fp = 'reviews_20000_tuned_cnn.p'
NAME = 'tune_20000'

CAT_COL = 'stars'

num_reviews = len(df.index)

train_num = int(num_reviews * .8)

train = df.iloc[:train_num]
validate = df.iloc[train_num:]

X_train = train['text'].values
y_train = to_categorical(train[CAT_COL].values)
print(y_train.shape)

X_validate = validate['text'].values
y_validate = to_categorical(validate[CAT_COL].values)
print(y_validate.shape)

max_length = max(max([len(review) for review in X_train]),  
              max([len(review) for review in X_validate]))

tokenizer = Tokenizer()

tokenizer.fit_on_texts(X_train)

num_words = len(tokenizer.word_index) + 1

X_train_tokens = tokenizer.texts_to_sequences(X_train)
X_validate_tokens = tokenizer.texts_to_sequences(X_validate)

X_train_pad = pad_sequences(X_train_tokens, maxlen=max_length, padding='post')
X_validate_pad = pad_sequences(X_validate_tokens, maxlen=max_length, padding='post')

# practitioners guide + that one article
#https://keras-team.github.io/keras-tuner/
def build_model(hp):
  global num_words
  model = Sequential()
  model.add(Embedding(num_words, 
                      output_dim=hp.Choice('embedding_output_dimension', values=[50, 100, 150], default=100),
                      input_length=max_length)) # experiment with size of vector space
  model.add(Conv1D(filters=hp.Choice('num_filters', values=[50, 100, 200, 400, 600],default=50),
                    kernel_size=hp.Int('conv_kernel', min_value=1, max_value=10, step=1), 
                    activation=hp.Choice('conv_activ', values=['relu', 'tanh'], default='relu'))) # experiement with filter / kernel size / activation fxn
  model.add(MaxPooling1D(pool_size=2))
  model.add(Flatten())
  model.add(Dropout(rate=hp.Float('drop_rate', min_value=0, max_value=.5, step=.1)))
  model.add(Dense(6, activation='softmax')) # This is a necessity
  # compile network
  model.compile(loss='categorical_crossentropy', optimizer='adam',  metrics=[keras.metrics.Precision(name='precision'),
                  keras.metrics.Recall(name='recall')])
  return model

tuner = RandomSearch(
          build_model,
           metrics=[keras.metrics.Precision(name='precision'),
             keras.metrics.Recall(name='recall')],
          objective=kerastuner.Objective('val_precision', direction='max'),
          max_trials=5,
          executions_per_trial=1,
          seed=10,
          directory='tuning_out',
          project_name=NAME)

tuner.search_space_summary()

tuner.search(X_train_pad, y_train, epochs=10, validation_data=(X_validate_pad, y_validate))

tuner.results_summary()

best_model = tuner.get_best_models(num_models=1)[0]
model_json = best_model.to_json()
open('reviews_20000_trained_cnn.json', 'w').write(model_json)
best_model.save_weights('reviews_20000_trained_cnn_weights.h5')

(16000, 6)
(4000, 6)


Epoch 1/10
500/500 [==============================] - 40s 81ms/step - loss: 1.1786 - precision: 0.5472 - recall: 0.3173 - val_loss: 0.8982 - val_precision: 0.6786 - val_recall: 0.4466
Epoch 2/10
500/500 [==============================] - 40s 81ms/step - loss: 0.7011 - precision: 0.7152 - recall: 0.5055 - val_loss: 0.9454 - val_precision: 0.7330 - val_recall: 0.5401
Epoch 3/10
500/500 [==============================] - 40s 80ms/step - loss: 0.3811 - precision: 0.7642 - recall: 0.5868 - val_loss: 1.1117 - val_precision: 0.7828 - val_recall: 0.6212
Epoch 4/10
500/500 [==============================] - 40s 80ms/step - loss: 0.1333 - precision: 0.8051 - recall: 0.6578 - val_loss: 1.4148 - val_precision: 0.8208 - val_recall: 0.6871
Epoch 5/10
500/500 [==============================] - 40s 81ms/step - loss: 0.0382 - precision: 0.8343 - recall: 0.7119 - val_loss: 1.7392 - val_precision: 0.8451 - val_recall: 0.7330
Epoch 6/10
500/500 [==============================] - 40s 81ms/step - loss: 0.01

Epoch 1/10
500/500 [==============================] - 16s 31ms/step - loss: 1.0666 - precision: 0.8812 - recall: 0.8074 - val_loss: 0.8972 - val_precision: 0.8757 - val_recall: 0.7926
Epoch 2/10
500/500 [==============================] - 15s 31ms/step - loss: 0.7677 - precision: 0.8712 - recall: 0.7836 - val_loss: 0.8814 - val_precision: 0.8667 - val_recall: 0.7749
Epoch 3/10
500/500 [==============================] - 15s 30ms/step - loss: 0.6499 - precision: 0.8639 - recall: 0.7695 - val_loss: 0.9460 - val_precision: 0.8605 - val_recall: 0.7645
Epoch 4/10
500/500 [==============================] - 15s 30ms/step - loss: 0.5109 - precision: 0.8591 - recall: 0.7626 - val_loss: 1.0565 - val_precision: 0.8570 - val_recall: 0.7603
Epoch 5/10
500/500 [==============================] - 15s 30ms/step - loss: 0.3479 - precision: 0.8573 - recall: 0.7612 - val_loss: 1.2071 - val_precision: 0.8569 - val_recall: 0.7619
Epoch 6/10
500/500 [==============================] - 15s 30ms/step - loss: 0.21

Epoch 1/10
500/500 [==============================] - 27s 54ms/step - loss: 1.1220 - precision: 0.8651 - recall: 0.7826 - val_loss: 0.9266 - val_precision: 0.8623 - val_recall: 0.7753
Epoch 2/10
500/500 [==============================] - 27s 54ms/step - loss: 0.7651 - precision: 0.8602 - recall: 0.7709 - val_loss: 0.8875 - val_precision: 0.8580 - val_recall: 0.7668
Epoch 3/10
500/500 [==============================] - 27s 54ms/step - loss: 0.5509 - precision: 0.8573 - recall: 0.7653 - val_loss: 0.9693 - val_precision: 0.8561 - val_recall: 0.7636
Epoch 4/10
500/500 [==============================] - 27s 54ms/step - loss: 0.2960 - precision: 0.8567 - recall: 0.7646 - val_loss: 1.2073 - val_precision: 0.8569 - val_recall: 0.7656
Epoch 5/10
500/500 [==============================] - 27s 54ms/step - loss: 0.1315 - precision: 0.8581 - recall: 0.7680 - val_loss: 1.4855 - val_precision: 0.8589 - val_recall: 0.7701
Epoch 6/10
500/500 [==============================] - 27s 53ms/step - loss: 0.06

Epoch 1/10
500/500 [==============================] - 19s 37ms/step - loss: 1.0525 - precision: 0.8655 - recall: 0.7853 - val_loss: 0.8885 - val_precision: 0.8639 - val_recall: 0.7804
Epoch 2/10
500/500 [==============================] - 18s 37ms/step - loss: 0.7599 - precision: 0.8626 - recall: 0.7773 - val_loss: 0.8689 - val_precision: 0.8611 - val_recall: 0.7742
Epoch 3/10
500/500 [==============================] - 18s 37ms/step - loss: 0.6273 - precision: 0.8601 - recall: 0.7724 - val_loss: 0.9310 - val_precision: 0.8589 - val_recall: 0.7704
Epoch 4/10
500/500 [==============================] - 19s 37ms/step - loss: 0.4675 - precision: 0.8585 - recall: 0.7699 - val_loss: 1.0444 - val_precision: 0.8579 - val_recall: 0.7692
Epoch 5/10
500/500 [==============================] - 19s 37ms/step - loss: 0.2954 - precision: 0.8582 - recall: 0.7699 - val_loss: 1.2239 - val_precision: 0.8582 - val_recall: 0.7704
Epoch 6/10
500/500 [==============================] - 18s 37ms/step - loss: 0.16

Epoch 1/10
500/500 [==============================] - 21s 41ms/step - loss: 1.0851 - precision: 0.8635 - recall: 0.7808 - val_loss: 0.8765 - val_precision: 0.8623 - val_recall: 0.7770
Epoch 2/10
500/500 [==============================] - 21s 41ms/step - loss: 0.7762 - precision: 0.8612 - recall: 0.7745 - val_loss: 0.8709 - val_precision: 0.8601 - val_recall: 0.7721
Epoch 3/10
500/500 [==============================] - 21s 41ms/step - loss: 0.6526 - precision: 0.8593 - recall: 0.7705 - val_loss: 0.8950 - val_precision: 0.8584 - val_recall: 0.7690
Epoch 4/10
500/500 [==============================] - 21s 41ms/step - loss: 0.5357 - precision: 0.8580 - recall: 0.7682 - val_loss: 0.9762 - val_precision: 0.8574 - val_recall: 0.7674
Epoch 5/10
500/500 [==============================] - 21s 41ms/step - loss: 0.4164 - precision: 0.8573 - recall: 0.7674 - val_loss: 1.0606 - val_precision: 0.8570 - val_recall: 0.7672
Epoch 6/10
500/500 [==============================] - 21s 41ms/step - loss: 0.32

INFO:tensorflow:Oracle triggered exit


In [4]:
max_length

3237

In [0]:
# Save the input vectors for later use
pickle.dump(X_train_pad, open('reviews_20000_X_train_pad.p', 'wb'))

In [0]:
# Prepate the test data
test = pickle.load(open('reviews_10000_test_processed.p', 'rb'))
X_test_tokens = tokenizer.texts_to_sequences(test['text'].values)
X_test_pad = pad_sequences(X_test_tokens, maxlen=max_length, padding='post')
pickle.dump(X_test_pad, open('reviews_20000_train_10000_test_pad.p', 'wb'))

In [0]:
from sklearn.metrics import classification_report
class_predictions = best_model.predict_classes(X_test_pad)
pickle.dump((class_predictions, test['stars']), open('reviews_20000_train_10000_test_cnn_class.p', 'wb'))
proba_predictions = best_model.predict_proba(X_test_pad)
pickle.dump((proba_predictions, test['stars']), open('reviews_20000_train_10000_test_cnn_proba.p', 'wb'))

test_report = classification_report(test['stars'], best_model.predict_classes(X_test_pad), output_dict=True)
train_report = classification_report(train['stars'], best_model.predict_classes(X_train_pad), output_dict=True)
open('cnn_20000_report.json', 'w').close()
print(test_report, file=open('cnn_20000_report.json', 'a'))
print(train_report, file=open('cnn_20000_report.json', 'a'))